<br><br><br>
<h2><font color="#004D7F" size=6>Práctica 3</font></h2>



<h1><font color="#004D7F" size=5> SISTEMAS BASADOS EN REGLAS DIFUSAS. ALGORITMO DE WANG Y MENDEL.</font></h1>

<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Javier Tomás Fernández Martin y Gabriel Chillerón Peinado</font><br>

<font color="#004D7F" size=3>Sistemas Basados en el Conocimiento</font><br>
<font color="#004D7F" size=3>Grado en Ingeniería Informática (UCLM)</font><br>
</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Introducción](#introduccion)
* [2. Metodología](#meto)
* [3. Implementación](#scikit)
* [4. Resultados](#c45)
* [5. Entrega](#entrega)


---

<br>

<a id="introduccion"></a>
# <font color="#004D7F"> 1. Introducción</font>

En esta práctica vamos a partir del Sistema Basado en Reglas implementado para la práctica anterior y vamos a mantener el motor de inferencia, pero a la hora de generar las reglas se realizará mediante el algoritmo Wang y Mendel. Ésto significa que ya no será necesario el árbol de clasificación de la práctica 1 para generar el sistema basado en reglas.



---

<br>

<a id="data"></a>
# <font color="#004D7F"> 2. Metodología</font>



### <font color="#004D7F"> 2.1 Generación de los conjuntos difusos</font>

Al contrario que en la práctica anterior, en esta ocasión no es necesario fuzzificar la base de datos. Sí es necesario, sin embargo, generar las variables difusas a partir de la base de datos, ya que se utilizarán para sacar el grado de pertenencia para cada posible regla. Para ello, se puede utilizar el método `fit` del `FuzzyEncoder` ya creado.

### <font color="#004D7F"> 2.2 Extracción de las reglas</font>

En este caso, se extraerá una regla por cada uno de las instancias existentes en la base de datos. A partir de esa regla, se comprobará la base de datos y se introducirá o no en el sistema basado en reglas. 

### <font color="#004D7F"> 2.3 Motor de inferencia</font>

Una vez extraídas las reglas, se usará el código utilizado en la práctica anterior para realizar la inferencia de una nueva instancia en el sistema basado en reglas.

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 3. Implementación</font>

Para esta práctica el alumno debe llevar a cabo la implementación de las tres etapas descritas en la metodología, con el objetivo final de crear un sistema de reglas difusas basado en Wang y Mendel para el problema de la predicción de la lluvia en Australia y ser capaz de realizar predicciones a partir de la inferencia.

Vamos a describir un poco más en detalle cada uno de estos pasos:



### <font color="#004D7F"> 3.1 Variables Difusas</font>

El primer paso será generar las variables difusas a partir de las variables de la base de datos. Para ello, se proporciona el esqueleto de una clase que os ayudará a estructurar las variables con los elementos necesarios para la transformación posterior de la base de datos. Para simplificar, se utilizarán conjuntos difusos triangulares.

Los atributos de la clase serán los siguientes:
- `universe`: El universo del discurso de la variable. **IMPORTANTE** `scikit-fuzzy` asume que el universo del discurso está ordenado para todas sus operaciones. Sin embargo, cuando extraemos una columna de la base de datos para generar la variable difusa no tiene por qué estar ordenada. Habrá que ordenarla para poder utilizar las funciones de `scikit-fuzzy` pero sin modificar la base de datos original.
- `memb_func`: Un diccionario con los grados de pertenencia de cada elemento del universo del discurso para cada uno de los conjuntos difusos que se generen de la variable difusa
- `labels`: Etiquetas de cada uno de los conjuntos difusos
- `name`: Nombre de la variable difusa

Las funciones de la clase serán las siguientes:
- `_membership_functions`: Función interna para generar el diccionario con las distintas funciones de pertenencia. A COMPLETAR
- `fuzzify`: Función que devuelve el mejor conjunto difuso para un valor dentro del universo del discurso. A COMPLETAR
- `plot`: Función que muestra de manera gráfica los conjuntos difusos de la variable. Ya viene dada.

Para utilizar la clase, habrá que calcular previamente los triángulos de los conjuntos difusos. Para ello, se utilizará la estructura `fuzzy_points`, que es diccionario con la definición de los triángulos y forma `{label: [start, peak, end]}` para cada triángulo.

Para generar esta estructura `fuzzy_points` se recomienda calcular las particiones de las variables de manera automática, ya sea calculando por anchura o por frecuencia. `Pandas` tiene funciones que podéis buscar para discretizar una variable tanto por anchura como por frecuencia, se recomienda utilizarlas.

In [1]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import scipy.interpolate as inter
import pandas as pd

class FuzzyVariable:
    def __init__(self, name, variable, fuzzy_points):
        new_var = variable.copy()
        self.universe = (new_var.sort_values(new_var.columns[0]))[new_var.columns[0]].unique() #Comprobar que es asi
        self.labels = fuzzy_points.keys()
        self.memb_func = self._membership_functions(variable, fuzzy_points)
        self.name = name
    
    def _membership_functions(self, variable, fuzzy_points):
        fuzz_dict = {}
        
        for label in self.labels:
            fuzz_dict[label] = fuzz.trimf(self.universe,fuzzy_points[label])
        return fuzz_dict
    
    def fuzzify(self, value):
        best = ''
        max_val = -1
        for label in self.labels: # Iterar por cada variable para ver la mejor
            # TODO: DEVOLVER EL CONJUNTO DIFUSO QUE MAYOR GRADO DE PERTENENCIA TENGA
            #interp_membership(x, xmf, xx)
            if fuzz.interp_membership(self.universe,self.memb_func[label], value) > max_val:
                max_val = fuzz.interp_membership(self.universe,self.memb_func[label], value)
                best = label
        return best

    def plot(self):
        fig, ax0 = plt.subplots(nrows=1)
        for label in self.labels:
            ax0.plot(self.universe, self.memb_func[label], linewidth=1.5, label=label)
        ax0.set_title(self.name)
        ax0.legend()
        plt.tight_layout()

In [2]:
from sklearn.base import BaseEstimator


# Ejemplo de la estructura básica de un algoritmo que hereda de BaseEstimator siguiendo la estructura de SciKit
class FuzzyEncoder(BaseEstimator):
    # constructor
    def __init__(self, sets=3, labels=['Low', 'Mid', 'High']):
        self.fuzzy_vars = {}
        self.sets = sets
        self.labels = labels

    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X):
        # TODO: APRENDER LAS VARIABLES DIFUSAS A PARTIR DE LOS DATOS
        columnas = X.columns
        for columna in columnas:
            if type(X[columna][0]) != str: 
                
                equal_width_points = self._equal_width(X[columna].to_numpy(), self.sets)
                fuzzy_points = self._expand_points(equal_width_points, self.labels)
                fuzz_var = FuzzyVariable(columna,pd.DataFrame(X[columna]),fuzzy_points)
                self.fuzzy_vars[columna] = fuzz_var

        return self
        
    # y transform para aplicar lo aprendido y transformar la base de datos creando una nueva
    def transform(self, X):
        new_X = X.copy()
        # TODO: MODIFICAR new_X CON LAS VARIABLES DIFUSAS APRENDIDAS
        for key in self.fuzzy_vars.keys():
            new_X[key] = new_X[key].apply(self.fuzzy_vars[key].fuzzify)
        return new_X
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def _equal_width(self, var, sets):
        try:
            cut = pd.cut(var, sets - 1)
            sol = [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        except ValueError: 
            cut = pd.cut(var, sets - 1, duplicates='drop')
            sol = [var.min()] + [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        sol[0] = var.min()
        return sol

    def _expand_points(self, fp, labels):
        ep = {}
        expanded_points = [fp[0]] + fp + [fp[-1]]

        for i in range(len(expanded_points) - 2):
            ep[labels[i]] = [expanded_points[i], expanded_points[i+1], expanded_points[i+2]]

        return ep

### <font color="#004D7F"> 3.2 Preprocesamiento</font>


Se ha de realizar el mismo preprocesamiento que en la práctica 1 con dos nuevas restricciones:
- La variable `Location` hay que eliminarla. Esto lo vamos a hacer para que el árbol que salga no ramifique demasiado
- Las variables de puntos cardinales hay que simplificarlas. Se han de quedar simplemente con los cuatro puntos cardinales principales (N, S, E, W)

In [3]:
# Preprocesamiento de la BBDD de la practica 1

df = pd.read_csv('weatherAUS.csv')
#Quitamos los valores nulos y las columnas Location y Date
del df['Location']
del df['Date']

for i in range (0,df.shape[1]):
    if type(df[(df.columns)[i]][0]) == str:
        df[(df.columns)[i]].fillna((df[(df.columns)[i]].mode())[0], inplace = True)
    else:
        df[(df.columns)[i]].fillna(df[(df.columns)[i]].mean(), inplace = True)

#Simplificamos los puntos cardinales
for elemento in df['WindGustDir'].unique():
    df.replace(elemento,elemento[0],inplace = True)



features = df.columns
attributes = features[:-1]
target = features[-1]
X = df[attributes]
y=df[target]
X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,13.4,22.900000,0.6,5.468232,7.611178,W,44.00000,W,W,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No
1,7.4,25.100000,0.0,5.468232,7.611178,W,44.00000,N,W,4.0,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No
2,12.9,25.700000,0.0,5.468232,7.611178,W,46.00000,W,W,19.0,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No
3,9.2,28.000000,0.0,5.468232,7.611178,N,24.00000,S,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No
4,17.5,32.300000,1.0,5.468232,7.611178,W,41.00000,E,N,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.400000,0.0,5.468232,7.611178,E,31.00000,S,E,13.0,11.0,51.0,24.0,1024.6,1020.3,4.447461,4.50993,10.1,22.4,No
145456,3.6,25.300000,0.0,5.468232,7.611178,N,22.00000,S,N,13.0,9.0,56.0,21.0,1023.5,1019.1,4.447461,4.50993,10.9,24.5,No
145457,5.4,26.900000,0.0,5.468232,7.611178,N,37.00000,S,W,9.0,9.0,53.0,24.0,1021.0,1016.8,4.447461,4.50993,12.5,26.1,No
145458,7.8,27.000000,0.0,5.468232,7.611178,S,28.00000,S,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.000000,2.00000,15.1,26.0,No


In [4]:
fuzzy_vars = FuzzyEncoder()
fuzzy_vars.fit(X)

FuzzyEncoder()

In [5]:
fuzzy_X = X
fuzzy_X = fuzzy_vars.transform(fuzzy_X)

In [6]:
fuzzy_X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Low,Mid,Mid,High,Mid,Mid,Mid,No
1,Mid,Mid,Low,Low,Mid,W,Mid,N,W,Low,Mid,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
2,Mid,Mid,Low,Low,Mid,W,Mid,W,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Low,Mid,Mid,No
3,Mid,Mid,Low,Low,Mid,N,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
4,Mid,Mid,Low,Low,Mid,W,Mid,E,N,Low,Low,High,Mid,Mid,Mid,High,High,Mid,Mid,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Mid,Mid,Low,Low,Mid,E,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145456,Mid,Mid,Low,Low,Mid,N,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145457,Mid,Mid,Low,Low,Mid,N,Low,S,W,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145458,Mid,Mid,Low,Low,Mid,S,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Low,Mid,Mid,No


Vamos a utilizar `scikit-fuzzy` para el sistema de control. Eso significa que se necesita un procesado previo para las variables discretas. Se necesitan dos elementos para crear un sistema basado en reglas con `scikit-fuzzy`:
- Universo del discurso. Para una variable discreta, será una enumeración de sus valores. Esto quiere decir que una variable `['N', 'S', 'E', 'W']` tendrá como universo del discurso `[0,1,2,3]`
- Una función de grados de pertenencia. En lugar de utilizar `fuzz.trapmf` o `fuzz.trimf`, la función será `1` cuando coincida con el elemento y `0` cuando no coincida. Para la variable anterior, 
  - `pert['N'] = [1,0,0,0]`,   
  - `pert['E'] = [0,0,1,0]`, 
  - etc.

Se recomienda crear un diccionario para cada una de las variables discretas para luego la generación de las reglas

In [7]:
def get_discrete(X, var):
    lusul = {}
    if type(X[var][0]) is str:
        lusul = {'universe': np.array(range(0,len(X[var].unique()))),
                'pert':{}
                }
        i = 0
        for label in X[var].unique():
            array = np.zeros(len(X[var].unique()),int)
            array[i] = 1
            lusul['pert'][label] = np.array(array)
            i = i+1    
    return lusul

Con este generador, dada la función `get_discrete` que nos genera las variables discretas, se consigue un diccionario por cada una de las variables discretas

In [8]:
columDiscretas = []
for var in X.columns:
    if type(X[var][0]) is str:
        columDiscretas.append(var)
discrete_variables = {att: get_discrete(fuzzy_X, att) for att in columDiscretas}

In [9]:
discrete_variables

{'WindGustDir': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir9am': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir3pm': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'E': array([0, 1, 0, 0]),
   'N': array([0, 0, 1, 0]),
   'S': array([0, 0, 0, 1])}},
 'RainToday': {'universe': array([0, 1]),
  'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}}

De igual manera, hay que crear un diccionario con la variable clase, que también es discreta. Se crea igual que el resto, pero se aconseja pasar esta variable por separado. 

Finalmente se crea el sistema basado en reglas difusas. Vamos a tener una clase abstracta general y una clase que extienda de esta y utilice Wang y Mendel para obtener las reglas.

La clase abstracta `RuleBasedSystem`, igual que en la práctica anterior, se dividirá en las funciones:
- `fit`: Metodo abstracto que se programará en la clase que extienda de `RuleBasedSystem`
- `_create_rbs`: encargado de aprender el sistema basado en reglas. Tiene los siguientes parámetros:
  - `rule_list`: Lista de reglas con las que se creará el sistema de control
  - `fuzzy_variables`: Variables difusas extraidas de `FuzzyEncoder.fuzzy_vars`
  - `discrete_variables`: Diccionario con las variables discretas.
  - `target_variable`: Diccionario con la variable objetivo.
- `predict`: Devuelve la clasificación para una instancia de la base de datos segun el sistema basado en reglas
- `score`: Devuelve el score de la base de datos

In [10]:
from skfuzzy import control as ctrl
from abc import ABC, abstractmethod

class RuleBasedSystem(ABC):
    def __init__(self):
        self.ctrl = None

    def _create_rbs(self, rule_list, fuzzy_variables, discrete_variables, target_variable):
        # TODO: PASO 1
        
        # Hay que obtener todos los antecedentes. Un antecedente en 
        # scikit-fuzzy se crea con ante = ctrl.Antecedent(universe, variable_name)
        # Para ello, tanto en fuzzy_variables como en discrete_variables
        # deberia estar ya representado el universo de cada variable

        # Ademas, para cada antecedente hay que crear todos los conjuntos difusos
        # (o valores para las variables discretas) correspondientes. Tenemos las etiquetas
        # en las fuzzy_variables, y se pueden obtener tambien de las variables discretas
        # Para cada conjunto difuso se crea con ante[label] = pert, donde ya tenemos
        # las funciones de pertenencia tanto en fuzzy_variables como en discrete_variables
        
        self.target = target_variable

        dict = {}
        for var in fuzzy_variables:
            dict[var] = ctrl.Antecedent(fuzzy_variables[var].universe,var)
            for label in fuzzy_variables[var].labels:
                dict[var][label] = fuzzy_variables[var].memb_func[label]
        
        for var in discrete_variables:
            dict[var] = ctrl.Antecedent(discrete_variables[var]['universe'],var)
            for label in discrete_variables[var]['pert'].keys():
                dict[var][label] = discrete_variables[var]['pert'][label]


        

        #rule_list = [rule1,rule2,rule3]
        #rule1 = (antecedentes, consecuente)
        #antecedentes = [ante1,ante2,ante3]
        #ante1 = [nombre,etiqueta]



        # TODO: PASO 2

        dict['target'] = ctrl.Consequent(target_variable['universe'],'RainTomorrow')
        for label in target_variable['pert'].keys():
            dict['target'][label] = target_variable['pert'][label]



        # Para crear el consecuente se hace con conseq = ctrl.Consequent(universe, conseq_name)
        # Por eso es mejor pasarlo por separado. De igual manera, hay que asignar cada uno 
        # de los valores que puede tomar y sus funciones de pertenencia






        rules = []

        # TODO: PASO 3

        # Finalmente hay que generar las reglas. Para ello se crean con 
        # rule = ctrl.Rule(antecedent=antecedents, consequent=consequent)
        # Aqui trabajamos con los antecedentes y el consecuente que hemos generado
        # anteriormente. Con la lista de reglas que tenemos del arbol, generamos
        # las reglas con el formato de scikit-fuzzy. antecedents es una union logica 
        # de terminos, esto quiere decir, ante1[label1] & ante2[label2]...
        # De igual manera, el consecuente es conseq[label] para el valor correspondiente
        # de la regla. Habra que haber almacenado los antecedentes para poder acceder
        # a ellos en funcion del nombre de la columna que aparece en la regla
        i = 0
        for rule in rule_list:
            
            antes = dict[rule[0][0][0]][rule[0][0][1]] #Inicializamos "antes" con la primera instancia
            for antece in rule[0]:

                if str(antece[0]+"["+antece[1]+"]") == str(antes):
                    pass
                else:
                    antes = antes & dict[antece[0]][antece[1]]
                
            rules.append(ctrl.Rule(antecedent = antes, consequent = dict['target'][rule[1]]))  
        
        
        
        
        
        self.ctrl = ctrl.ControlSystem(rules=rules)


    @abstractmethod
    def fit():
        pass
    
    def predict(self, x):
        sim = ctrl.ControlSystemSimulation(self.ctrl, flush_after_run=1000)
        # Para predecir, hay que añadir los valores de la instancia al sistema
        # de simulacion de control. Primero hay que ver si existe el antecedente 
        # en el sistema de control (posiblemente las reglas extraidas del arbol
        # no tengan todas las caracteristicas de la instancia). En caso afirmativo,
        # la variable 'temperatura' = 8.7 se añade como sim.input['temperatura'] = 8.7    
        for col in x.columns:
            for rule in self.ctrl.rules.all_rules:
                for ante in rule.antecedent_terms:
                    if str(col) in str(ante):
                        if type(x[col].unique()[0]) is str:
                            sim.input[col] = sum(discrete_variables[col]['pert'][x[col].unique()[0]] * discrete_variables[col]['universe'])
                        else:
                            sim.input[col] = x[col].unique()[0]
                        

        # TODO: IMPLEMENTAR

        
        # Despues se calculan los grados de las reglas
        sim.compute()

        # Finalmente se devuelve el resultado que se encuentra en sim.output['target']
        # si la variable a predecir se llama 'target'.
        
        # TODO: IMPLEMENTAR

        return sim.output['RainTomorrow']
    

    def score(self, X, y):
        all_predictions = np.array([round(self.predict(x)) for x in X])
        return np.sum(all_predictions==y)/y.shape[0]

La clase `WangMendelRuleBasedSystem` se dividirá en las funciones:
- `_extract_rules`: encargado de procesar la bbdd. Se proporciona un formato sugerido para extraer las reglas pero se puede utilizar el formato preferido.
- `fit`: Metodo que entrena el sistema basado en reglas

In [11]:
class WangMendelRuleBasedSystem(RuleBasedSystem):
    def __init__(self):
        super().__init__()

    # Se recomienda este formato para extraer las reglas
    # del arbol, ya que simplifica la manera de tratar los 
    # antecedentes y consecuentes a la hora de generar el 
    # sistema basado en reglas
    def grado(self,var,value):
        max_val = 0
        best = ""
        for label in var.labels:
            if fuzz.interp_membership(var.universe,var.memb_func[label], value) > max_val:
                max_val = fuzz.interp_membership(var.universe,var.memb_func[label], value)
                best = label
                
        return max_val, best      
          
    
    def _extract_rules(self, X, y, fuzzy_variables):

        # Para el algoritmo de Wang y Mendel, que se debe realizar en esta funcion
        # Se recomienda el uso de un diccionario del tipo {rule: membership}
        # para que a la hora de almacenar las reglas y comprobar si hay que actualizar
        # el grado de pertenencia, sea lo mas eficiente posible.
        # Recordad que para que sea compatible con el algoritmo de la practica anterior,
        # habra que transformar ese diccionario a una lista antes de devolverlo.

        # fuzzy_variables sera un subconjunto de las variables difusas del problema, 
        # ya que un problema con tantas instancias y variables puede resultar muy 
        # complejo de extraer todas las reglas
        
        
        all_rules = {}
        dicc = {}
        for i in range(len(X)):
            finalgrado = 1
            antecedentes = [] 
            j = 0
            for val in X.loc[i]:
                if X.columns[j] in fuzzy_variables: 
                    if type(val) != str:
                        (auxgrado,aux) = (self.grado(fuzzy_variables[X.columns[j]],val))
                        antecedentes.append((X.columns[j],aux))
                        finalgrado *= auxgrado
                    else:
                        antecedentes.append((X.columns[j],val))
                    j = j+1
                rule = (antecedentes,y.loc[i])
                if str(antecedentes) in dicc.keys():
                    if finalgrado > dicc[str(antecedentes)]:
                        dicc[str(antecedentes)] = finalgrado
                        all_rules[str(antecedentes)] = rule
                else:
                    dicc[str(antecedentes)] = finalgrado
                    all_rules[str(antecedentes)] = rule
                
         
        final_rules = []    
        for key in all_rules.keys():
            final_rules.append(all_rules[key])

        

        return final_rules


    def fit(self, X, y, fuzzy_variables, discrete_variables, target_variable):
        rule_list = self._extract_rules(X, y, fuzzy_variables)
        self._create_rbs(rule_list, fuzzy_variables, discrete_variables, target_variable)


In [12]:
discrete_target = get_discrete(pd.DataFrame(y),"RainTomorrow")
print(discrete_target)

{'universe': array([0, 1]), 'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}


In [13]:
subconj = {}
i = 0
#Para cambiar el subconjunto de variables difusas, modificar el nsubconjuntos
nsubconjuntos = 4
for aux in fuzzy_vars.fuzzy_vars.keys():
    if nsubconjuntos == i :
        break
    else:
        subconj[aux] = fuzzy_vars.fuzzy_vars[aux]
        i +=1
        
subconj

{'MinTemp': <__main__.FuzzyVariable at 0x1cb33bd8c10>,
 'MaxTemp': <__main__.FuzzyVariable at 0x1cb32d2dac0>,
 'Rainfall': <__main__.FuzzyVariable at 0x1cb32d2d3a0>,
 'Evaporation': <__main__.FuzzyVariable at 0x1cb32d2d700>}

In [25]:
#Hemos cogido solo 100000 valores para poder hacer el predict con los restantes.
#Tarda aproximadamente 1 minuto en sacar todas las reglas.
wang = WangMendelRuleBasedSystem()
pruebax = X[0:100000]
yprueba = y[0:100000]
wang.fit(pruebax,yprueba,subconj,discrete_variables,discrete_target)

In [30]:
wang.predict(X.loc[1454:1454])

0.47321991178323886

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 4. Resultados</font>

Para finalizar esta práctica, el alumno deberá ejecutar el algoritmo con distintos conjuntos de variables difusas, y compararlo tanto en score como en tiempo de ejecución con el algoritmo de la práctica anterior. Habrá que hacer comparativas mínimo con conjuntos de 3, 4 y 5 variables difusas.

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 5. Entrega</font>

Al igual que en la práctica anterior, el alumno subirá a campus virtual un notebook con la implementación del sistema de reglas y del proceso de inferencia.

Para aprobar la práctica será necesario que la implementación del algoritmo sea correcta. Se tendrán en cuenta para la nota la eficiencia de la implementación, así como la limpieza y los comentarios en el código.

La parte de resultados será necesaria llevarla a cabo para optar a la nota máxima.

La entrega de esta práctica tiene de plazo hasta el día 5 de Junio de 2022.